# Basic Finesse Modeling of a Fabry Perot Cavity

This notebook makes a simple Fabry Perot cavity in Finesse and computes the frequency response and response to sweeping drives. State space and zpk representations of the optomechanical plant are computed at the end.

[__1.__](#model) Model definition

[__2.__](#frequency-response) Compute the frequency response

[__3.__](#sweep) Compute the response to sweeping drives

[__4.__](#ss-zpk) Find state space and zpk representations of the optomechanical plant

The BasicOptickleFP notebook goes through the identical calculations with Optickle.

In [ ]:
import numpy as np
import pytickle.finesse as fin
import pykat
from pytickle.plotting import plotTF
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.constants as scc
%matplotlib inline

In [ ]:
mpl.rc('figure', figsize=(12, 9))

mpl.rcParams.update({'text.usetex': False,
                     'mathtext.fontset': 'cm',
                     'lines.linewidth': 3,
                     'lines.markersize': 10,
                     'font.size': 16,
                     'axes.grid': True,
                     'grid.alpha': 0.5,
                     'legend.loc': 'best',
                     'savefig.dpi': 80,
                     'pdf.compression': 9})

<a name="model"> </a>

## Model Definition

PyTickle has several functions that make building a Finesse model more convenient and similar to Optickle model building. These functions just make a regular PyKat kat model. Any kat model made by any means outside of PyTickle, for example by using PyKat to parse classic Finesse code, can be used with its analysis functions.

Most Finesse model building functions have analogs in PyTickle. Just as in Finesse, components are added with nodes and the nodes are connected with spaces. Using PyTickle's model building functions enforces standardized node naming conventions which the user does not have to think about. One consequence is that the behavior of the model will never depend on the order in which components are defined or in which spaces are added.

For example, the `addMirror(kat, 'mirr')` function adds a mirror named `'mirr'` to the model `kat` with a set of default parameters and defines the nodes `mirr_fr` and `mirr_bk` for the front and back of the mirror, respectively. Most functions have doc strings, so the usage, default parameters, and node names can be found easily using, for example, `help(fin.addMirror)`.

In [ ]:
# define some parameters
fmod = 11e6  # modulation frequency for PDH sensing [Hz]
gmod = 0.1   # modulation depth
Pin = 1      # input power [W]
Ti = 0.014   # input coupler transmissivity
Lcav = 40e3  # cavity length [m]

# start a new model
kat = pykat.finesse.kat()

# make the cavity
fin.addMirror(kat, 'EX')                   # add a perfectly reflecting mirror
fin.addMirror(kat, 'IX', Thr=Ti)           # add a mirror with transmissivity Ti
fin.addSpace(kat, 'IX_fr', 'EX_fr', Lcav)  # connect the front faces to form a cavity

# add input
fin.addLaser(kat, 'Laser', Pin)
fin.addModulator(kat, 'Mod', fmod, gmod, 1, 'pm')  # RF modulator for PDH sensing
fin.addSpace(kat, 'Laser_out', 'Mod_in', 0)
fin.addSpace(kat, 'Mod_out', 'IX_bk', 0)

# add DC and RF photodiodes
fin.addReadout(kat, 'REFL', 'IX_bk', fmod, 5)

Note that mirrors and beamsplitters in Finesse are fake single surface objects which PyTickle's `addMirror` and `addBeamsplitter` functions add by default. Real mirrors with both an HR and AR side can be added by setting the `comp=True` keyword in the function calls. The node names which the user should use are the same as when `comp=False`.

<p>&nbsp;</p>

`addReadout` is a convenience function that adds RF and DC probes to a detection port. So the last command above
```python
fin.addReadout(kat, 'REFL', 'IX_bk', fmod, 5)
```
added three probes to the back of the mirror `IX`:
 1. A DC photodiode named `REFL_DC`
 2. An RF photodiode named `REFL_I` demodulated at frequency `fmod` with phase 5.
 3. An RF photodiode named `REFL_Q` demodulated at frequency `fmod` with phase 5 + 90 = 95.

<a name="frequency-response"> </a>

## Frequency Response

To compute transfer functions from a kat model, create a `KatFR` object from that model.
```python
katFR = fin.KatFR(kat)
```
By default, the response of a Finesse model to all drives is computed. With complex models this can take some time. The computation time can be reduced if only a subset of the drives will be needed for further computations by
```python
katFR = fin.KatFR(kat, all_drives=False)
katFR.addDrives(drive_list)  # add with a list
katFR.addDrives('EX')        # or add additional drives one at a time
```

Calling
```python
katFR.run(fmin, fmax, npts)
```
does the actual calculation for the frequency vector from `fmin` to `fmax` with `npts` points. By default two things are computed, but this can be controlled with the `rtype` keyword:
1. The AC transfer functions from drives to probes, i.e. the optomechanical plant. Using `rtype='opt'` will only compute this.
2. The radiation pressure modifications to the mechanical response of the drives, i.e. the "radiation pressure loop suppression function". This is explained in the torsional spring example and in detail in the control system example, but we do not need it here. Using `rtype='mech'` will only compute this.
3. By default `rtype='both'` and both of these are computed. Note that the radiation pressure effects are still calculated if `rtype='opt'` but the mechanical effects cannot be analyzed separately in this case.

The frequency vector used is
```python
katFR.ff
```

In [ ]:
katFR = fin.KatFR(kat)  

In [ ]:
# compute the AC response matrix, i.e. the optical plant
fmin = 1e-1
fmax = 10e3
npts = 1000
katFR.run(fmin, fmax, npts, rtype='opt')

After running the model, transfer functions from any drives `drives` to any probes `probes` can be calculated with
```python
katFR.getTF(probes, drives)
```
The variables `probes` and `drives` can be strings specifying the probes and drives or they can be dictionaries specifying linear combinations of probes and drives. For example,
```python
katFR.getTF('AS_Q', 'EX')
```
computes the respone at the probe `AS_Q` to motion of the mirror `EX`, while
```python
katFR.getTF('AS_Q', {'EX': 1/2, 'EY': -1/2})
```
computes the response at the probe `AS_Q` to the differentional motion of the mirrors `EX` and `EY` moving 180 degrees out of phase.

<p>&nbsp;</p>

The convenience function `plotTF` directly plots transfer functions. The optional third and fourth arguments are existing magnitude and phase axes so that multiple functions can be plotted on the same plot.

In [ ]:
fig = katFR.plotTF('REFL_I', 'EX', label='REFL_I')
katFR.plotTF('REFL_Q', 'EX', fig.axes[0], fig.axes[1], ls='-.', label='REFL_Q');
fig.axes[0].legend()
fig.axes[0].set_title('Response to EX Motion')
fig.set_size_inches((8, 11));

#### Saving and loading optomechanical plants and radiation pressure modifications for future use

Note also that the results of a simulation can be exported to an hdf5 file and loaded for future analysis. This is useful in complex models that take a long time to run since they only need to be calculated once and can then be analyzed in the future without having to do the calculations again. Since no simulations are needed for previously run data, Finesse does not need to be installed to analyze previously run data. The exported hdf5 file can then be compared with an Optickle simulation by a user who does not have Finesse. Similarly, simulations run with Optickle can be exported and comparred with a Finesse simulation by a user who does not have Optickle or MATLAB.

For example, to save a model to the file `'pdh_freq_resp.hdf5'`
```python
katFR.save('pdh_freq_resp.hdf5')
```
and to load it back in a future script
```python
import pytickle.plant as plant
katFR = plant.FinessePlant()
katFR.load('pdh_freq_resp.hdf5')
```
All analysis functions can be done on this new `katFR` object created from a previously computed optomechanical plant, but the underlying Finesse `kat` model does not exist anymore so no further Finesse simulations can be run with it.

To use a model calculated by Optickle
```python
opt = plant.OpticklePlant()
opt.load('pdh_freq_resp_optickle.hdf5')
```
The analysis functions on this `opt` object are almost identical to those of the `katFR` object and can be analyzed without Optickle or MATLAB installed.

<a name="sweep"> </a>

## Sweeping Drives

To compute the response of a model to sweeping drives, create a `KatSweep` object from that model.
```python
katSweep = fin.KatSweep(kat, drives)
```
where `drives` are the drives to be swept. This can be a string specifying a drive or a dictionary specifying a linear combination of drives
```python
kat1 = fin.KatSweep(kat, 'EX')                     # sweep EX
kat2 = fin.KatSweep(kat, {'EX': 1/2, 'EY': -1/2})  # sweep EX - EY
```

Calling
```python
katSweep.sweep(spos, epos, npts)
```
does the actual calculation sweeping the drives from `spos` to `epos` in `npts` points.

By default the drives are swept around their operating point, but this can be controlled with the `relative` keyword when defining the `KatSweep` object. For example, suppose that the microscopic tuning of the drive `EX` has been set to 90 degrees with `kat.EX.phi = 90`. Then
```python
katSweep = fin.KatSweep(kat, 'EX', relative=True)  # default
katSweep.sweep(-10, 10, 100)
```
sweeps the drive `EX` from 80 deg to 100 deg, while
```python
katSweep = fin.KatSweep(kat, 'EX', relative=False)
katSweep.sweep(-10, 10, 100)
```
sweeps the drive `EX` from -10 to 10 deg.

In [ ]:
katSweep = fin.KatSweep(kat, 'EX')

# sweep from -5 nm to 5 nm
xf = 5e-9 * 360/kat.lambda0  # final position [deg]
xi = -xf                     # initial position [deg]
katSweep.sweep(xi, xf, 1000)

After running the model, the sweep signals are computed with
```python
poses, sig = katSweep.getTF(probe, drive)
```
This returns the signal `sig` as measured by `probe` as a function of the drive `drive` positions `poses`.

An optional third argument applies a function to the signal before returning it. Finesse returns complex signals, so even though a signal is real, the numbers are returned with zero imaginary part. We can get the real signal with
```python
poses, sweepI = katSweep.getSweepSignal('REFL_I', 'EX', np.real)
```
In our example, `poses` will be the linearly spaced vector of positions from `xi` to `xf` and `sweepI` will be the real signal measured in `REFL_I`.

As another example, if a model had the amplitude detector `amp_f1` to measure the amplituded of a sideband, the amplitude of this (complex) signal would be
```python
poses, amp = katSweep.getSweepSignal('amp_f1', 'EX', func=np.abs)
```
and the power would be
```python
poses, power = katSweep.getSweepSignal('amp_f1', 'EX', func=lambda x: np.abs(x)**2)
```
Of course the signals can be manipulated at will after being calculated; this is just for convenience.

In [ ]:
# get the error signals in REFL_I and REFL_Q
poses, sweepI = katSweep.getSweepSignal('REFL_I', 'EX', np.real)
_, sweepQ = katSweep.getSweepSignal('REFL_Q', 'EX', np.real)

In [ ]:
# compute the slope of the error signals
nn = int(len(poses)/2)  # index of the center x0 of the sweep
nh = nn + 2             # index of x0 + dx
nl = nn - 2             # index of x0 - dx
dx = poses[nh] - poses[nl]
dI = (sweepI[nh] - sweepI[nl]) / dx
dQ = (sweepQ[nh] - sweepQ[nl]) / dx

`plotSweepSignal` is another convenience function, like `plotTF`, which plots sweeps directly. The optional fourth argument is an existing figure so that multiple signals can be plotted on the same plot.

In [ ]:
fig = katSweep.plotSweepSignal('REFL_I', 'EX', np.real, label='REFL_I')
katSweep.plotSweepSignal('REFL_Q', 'EX', np.real, fig, label='REFL_Q')
ax = fig.gca()

# plot the error signal slopes
ymin, ymax = ax.get_ylim()
ax.plot(poses, poses*dI, 'C3:', label='dI/dx');
ax.plot(poses, poses*dQ, 'C2:', label='dQ/dx');
ax.set_ylim(ymin, ymax)

ax.legend()
ax.set_xlabel('EX position [deg]')
ax.set_ylabel('Power [W]');

<a name="ss-zpk"> </a>

## State Space and ZPK Representations

If you have IIRrational installed, you can find state space and zpk representations of the optomechanical plant.

In [ ]:
from pytickle.iirrational import PlantFit

This is done by fitting the plant to a rational function and finding the zeros, poles, and gain.
```python
plant = PlantFit(ff, plant_data)
```

We'll fit the `REFL_I` frequency response to `EX` motion. This may take a few minutes. You can also pass any IIRrational `data2filter` keyword arguments to `PlantFit`. PyTickle sets the initial fit order to be 20, but this can be tuned with the `order_initial` keyword by
```python
plant = PlantFit(ff, plant_data, order_initial=10)  # start fitting with order 10
```
In this case, starting with 20 happens to be relatively fast and gives a good fit, but the behavior can vary with different plants.

In [ ]:
# get data
tfI = katFR.getTF('REFL_I', 'EX')  # transfer function to REFL_I
ff = katFR.ff                      # frequency vector

In [ ]:
# fit the plant
plant = PlantFit(ff, tfI)

Once the plant has been fit, IIRrational's `investigate_order_plot` function can be used to investigate the various fit orders.

In [ ]:
plant.investigate_order_plot();

Particular fits are chosen with `plant.choose(order)`. After choosing a fit order, IIRrational's `investigate_fit_plot` function can be used to investigate this fit in detail. The 5th order fit looks like the best one that has not been overfit in this case.

In [ ]:
plant.choose(5);  # choose the 5th order fit
plant.investigate_fit_plot();

We recognize some of the poles for a Fabry Perot cavity that we know should be here
$$p_n = f_p \pm \mathrm{i} n f_\mathrm{fsr}$$
where $f_p = cT/8\pi L$ is the cavity pole and $f_\mathrm{fsr} = c/2L$ is the FSR. The full plant has an infinite number of poles, but using only the poles $[-n, n]$ is a good approximation for frequencies $f\lesssim nf_\mathrm{fsr}/2$. We have computed two FSRs and it thus not surprising that the best fit is the 5th order one and that the poles are close to the theoretical values for $n\in [-2, 2]$.

In [ ]:
fp = scc.c*Ti/(8*np.pi*Lcav)
fsr = scc.c/(2*Lcav)
zs, ps, k = plant.get_zpk(Hz=True)
print('Cavity pole: {:0.1f} Hz'.format(fp))
print('FSR: {:0.1f} Hz'.format(fsr))
print('Fitted poles:')
for pole in ps:
    print('{:0.1f} + {:0.1f}i Hz'.format(pole.real, pole.imag))

After a satisfactory fit is found, the zpk representation is found with
```python
zs, ps, k = plant.get_zpk()
```
which returns the zeros `zs` and poles `ps` as arrays in the s-domain. Using
```python
zs, ps, k = plant.get_zpk(Hz=True)
```
returns instead zeros and poles as real frequencies in Hz. This can be converted to a scipy.signal state space representation by
```python
ss = plant.get_state_space()
```

In [ ]:
ss = plant.get_state_space()